In [188]:
import pandas as pd 
issues = pd.read_csv("issues.csv", sep=";", encoding='cp1252', names=['issue_id', 'assigned_person', 'title', 'description', 'parent issue', 'time created', 'closed_on'])
comments = pd.read_csv("comments.csv", sep=";", encoding='cp1252', names=['id', 'issue', 'person', 'comment', 'time'])
people = pd.read_csv("people.csv", sep=";", encoding='cp1252', names=['number', 'first', 'last'])
attachments = pd.read_csv("attachments.csv", sep=";", encoding='cp1252', names=['filename', 'diskname', 'user_id', 'date', 'month', 'issue', 'description'])

### Combines names for concision

In [189]:
people['name'] = 'assigned to: ' + people['first'] + ' ' + people['last']

## We have imported the issues database (created when an issue is created) and the comments (a seperate database that holds comments related to each issue), as well as People, which shows who corresponds to which number in the database and Attachments for any relevant file attachments. For attachments, we will not be able to transfer these directly to Gitlab, but can reference them for clarity when viewing old Issues

## The import of the sql dump has a lot of issues, like randomly adding words in the "issue_id" section. This gets rid of those by checking if the id is numeric, and eliminating the row if it is not. This removes ~1/3 of the databases rows, of which are all nonsense

In [190]:
for i, j in issues.iterrows(): 
    k = j['issue_id']
    if k.isnumeric():
        pass
    else:
        issues = issues.drop(labels=i, axis=0)

## This will match people with their number as referenced in the MySQL database

In [191]:
people['number'] = people['number'].astype(str)

In [192]:
attachments['user_id'] = attachments['user_id'].astype(str)

In [193]:
# changes names for both the People and Attachments dataframes
for i, j in people.iterrows():
    k = j['number']
    issues.loc[issues['assigned_person'] == k, 'assigned_person'] = j["name"]
    
for i, j in people.iterrows():
    k = j['number']
    attachments.loc[attachments['user_id'] == k, 'user_id'] = j['name']

## Formats the time that comments and attachments were created

In [194]:
attachments['date'] = attachments['date'].str[:15]
comments['time'] = comments['time'].str[:19]

## Sometimes a person is not assigned to an issue for some reason. I could not find any logic in why, so I just replaced with an error message

In [195]:
issues["assigned_person"] = issues["assigned_person"].replace(['\\N'], 'ERROR: person not found')

## Can use Gitlab QuickActions to assign people to an issue (and many other functionalities), so I do that here. When uploaded, putting "/assign @name" in the description will asign it to the username that follows the @. Since a lot of this is historical record keeping, if the user is not at TechX or doesnt have access to our Gitlab, it will remain as "assigned to: *user*"

In [196]:
issues["assigned_person"] = issues["assigned_person"].replace(['assigned to: *Person*'], '/assign @GITLAB_USERNAME')
# Repeat this for every person in Gitlab

## For some reason time sometimes isnt found. This fills in that NaN error and also fills in for when no parent issue is assigned, or when an issue is still open.

In [197]:
issues['time created'] = issues['time created'].fillna("time not found")
issues['parent issue'] = issues['parent issue'].fillna("no parent issue")
issues['closed_on'] = issues['closed_on'].fillna('1') # if closed, substituted w/ 1.

## Here I begin formatting to make data more readable

In [198]:
issues["description"] = issues["assigned_person"] + "\ntime created: " + issues["time created"] + "\n" + issues["description"] 

## Reverses the comments dataframe so that they are in order when I attach the comments to the issues

In [199]:
comments = comments.iloc[::-1] 

In [200]:
comments.reset_index(drop=True, inplace=True)

## Attachments are not included at all. They are in a separate file system and database, and are not referenced in the databases used up to this point. Here I will structure them and add them to the relevant issues/comments

### To access those, you will have to have go to Redmine or have access to the Redmine file system.

### Creating attachments column to put in the Issues dataframe

In [201]:
issues.insert(0, "attached_files", "")

In [202]:
attachments['description'] = attachments['description'].fillna('')

## Attaches the attachment to its associating issue and describes how to access it, while also showing the user  and the filename

In [203]:
for i, j in attachments.iterrows():
    description = 'Find attachments at your Redmine by searching the title of this Issue\nrelevant attachments: ' + 'user: ' + j['user_id'] + ' '
    row = issues.index[j["issue"] == issues["issue_id"]].tolist()
    if j['description']:
        description = description + "; attachment description: " + j['description']
    if not row:
        pass
    else:
        for i in range(len(row)):
            issues.at[row[i], "attached_files"] = issues.at[row[i], "attached_files"] + description + "; file name: " + j["filename"]

## Comments "id" section have the same issue as the titled issues; random id's become interrupted by nonsense words. This drops those rows.

## The Issues data had some missing descriptions that were only given titles. We kept these because that could be an intentional input, creating an issue with no description. But for comments, theres a lot of empty comments due to different updates, and this begins the process of dropping all those

In [204]:
for i, j in comments.iterrows(): 
    k = j['id']
    if k.isnumeric():
        pass
    else:
        comments = comments.drop(labels=i, axis=0)

for i, j in issues.iterrows():
    desc = j['description']
    if type(desc) == str:
        pass
    else:
        issues["description"] = issues["description"].fillna("")

## Replaces peoples numbers with their names just like I did above in the Titled issues

In [205]:
for i, j in people.iterrows():
    k = j["number"]
    comments.loc[comments["person"] == k, 'person'] = j["name"]

In [206]:
comments["person"] = comments["person"].replace(['assigned to: *Person*'], '@GITLAB_USERNAME')
# Repeat for every person in your Gitlab

## Cleaning data
### the sql dump seems to have a lot of issues that won't resolve, like missing data (namely "time") and empty comments due to status updates of an issue (ex. changing from 20% done to 30% done). This next block gets rid of them all

In [207]:
for i, j in comments.iterrows():
    k = j['comment']
    p = j['time']
    
    # gets rid of escape characters, that arise with status changes like "new" to "resolved" in redmine
    if k == "\\N":
        comments = comments.drop(labels=i, axis=0)
        
    # Checks if comment is "NaN" (empty), drops it if so
    elif type(k) == str:
        pass
    else:
        comments = comments.drop(labels=i, axis=0)
        
    # checks if time is "NaN". No solution for this unfortunately, but need to keep the comment
    if type(p) == str:
        pass
    else:
        comments = comments.replace([p], "time not found")
    

# Beginning formatting of the comments to be attached to the issues.

In [208]:
comments["full_comment"] = "\n\n" + "User: " + comments["person"] + "\ntime & date: " + comments["time"] + "\ncomment: " + comments["comment"]

### Iterates through the comments and then appends it to the corresponding issue

In [209]:
for i,j in comments.iterrows():
    row = issues.index[issues["issue_id"] == j["issue"]].tolist()
    issues.at[row[0], "description"] = issues.at[row[0], "description"] + j["full_comment"]
    
for i, j in issues.iterrows():
    j['description'] = j['description'] + j['attached_files']

In [214]:
for i, j in issues.iterrows():
    if j['closed_on'] != '\\N':
        j["description"] = "Issue was closed on: " + j["closed_on"] + "\n\n" + j["description"]

In [215]:
issues = issues.drop(['attached_files', 'issue_id', 'assigned_person', 'parent issue', 'time created', 'closed_on', 'parent_epic'], axis=1)

In [ ]:
issues.to_csv("filename", sep=',')